# <center> Feature engineering: TSFresh performance

In [13]:
%store -r __ipy
%store -r __da

In [5]:
__ipy

Helper ipython script loaded


In [6]:
__da

Basic Data Analysis tools was loaded


In [12]:
import matplotlib.pyplot as plt
import lightgbm as lgb
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
import numpy as np
import pandas as pd
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters ,ComprehensiveFCParameters
from sklearn.model_selection import KFold, cross_validate, cross_val_predict
import timeit
import seaborn as sns
import sys
import time
from tqdm import tqdm
import warnings
from tsfresh.utilities.dataframe_functions import make_forecasting_frame

In [8]:
settings={
'standard_deviation': None,
 'variance': None,
 'skewness': None,
 'kurtosis': None,
 'absolute_sum_of_changes': None,

'last_location_of_maximum': None,
 'first_location_of_maximum': None,
 'last_location_of_minimum': None,
'first_location_of_minimum': None,
'count_above_mean': None,
 'count_below_mean': None,
 'maximum': None,
 'minimum': None,

 'number_peaks': [{'n': 1}, {'n': 5}],
 'ar_coefficient': [{'coeff': 0, 'k': 10}]
}

In [9]:
train_df = pd.read_pickle(root + "train_features.pkl")

In [10]:
n_samples = 100000
train_t = train_df[:n_samples]

def tsfresh_time(sett, column_sort, column_value):
    for f, param in tqdm(sett.items()):
        res.loc[f, "feature"] = f
        fc_dict = {f:param}
        t = timeit.timeit(lambda : extract_features(train_t, 
                         column_id=KEY_NAME,
                         column_sort=column_sort,
                         column_value=column_value,
                         n_jobs=12, 
                         default_fc_parameters=fc_dict, 
                         disable_progressbar=True), 
                         number=3)
        
        res.loc[f, "t_abs"] = t
    return res

In [11]:
res=pd.DataFrame()
res=tsfresh_time(settings, 'timestamp', 'meter_reading')
res["feature"] = res.feature.astype(str)
res = res.sort_values(by="feature")

plt.figure(figsize=(6, 20))
sns.barplot(y="feature", x="t_abs", data=res)
plt.title("Runtime of 1 apply features for 1 time series of length %d"  % n_samples)
plt.show()

  0%|                                                                                                                                                                                                                                               | 0/15 [00:00<?, ?it/s]


NameError: name 'KEY_NAME' is not defined